In [208]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat

In [209]:
spamTrain_sample = loadmat('spamTrain.mat')
X = spamTrain_sample['X'] # (4000, 1899)
y_label = spamTrain_sample['y'] # (4000, 1)

some_num = 2000

X = X[0:some_num, :]
y_label = y_label[0:some_num, :]
print(X.shape)
print(y_label.shape)
y_label = y_label.reshape(y_label.shape[0])
print(y_label.shape)


(2000, 1899)
(2000, 1)
(2000,)


In [210]:
C = 0.01
k = 2

b = cp.Variable(1)
n_feat = X.shape[1]
n_train = X.shape[0]
# M = X.T
y_label = np.int_((y_label - 0.5)*2)
M = np.multiply(X, y_label[:, np.newaxis])
M=M.T
print(M.shape)

w = cp.Variable(n_feat)
zeta = cp.Variable(n_train)

# print(y_label)

objective_soft = cp.Minimize(0.5*cp.norm(w,2)**2 + C*(cp.sum(zeta))**k)
constraints_soft = [
    zeta >= 0,
    cp.multiply(y_label, (w.T@M + b)) >= 1 - zeta,
]
problem_soft = cp.Problem(objective_soft, constraints_soft)
problem_soft.solve(solver=cp.SCS)

(1899, 2000)


1.0104542778396621

In [211]:
print(w.value.shape)
print(w.value)
print(w.value[1])
print(b.value)
print(np.round(zeta.value,3))
# i1 = 0
# i2 = 1
# x = np.linspace(-30, 30, 100)
# print(-(w.value[i1]*x + b.value)/w.value[i2])

(1899,)
[2.75670684e-08 3.92975773e-08 2.56936352e-07 ... 1.14000738e-07
 1.10476546e-07 1.80124762e-07]
3.929757733279645e-08
[-0.00111174]
[0. 0. 0. ... 0. 0. 0.]


In [212]:
def accuracy_Fucked(y_label, X, w, b):
    y_pred = w@X+b > 0
    y_pred = 1*y_pred
    # print(y_pred)
    return np.mean(y_label == y_pred)

In [213]:
spamTest_sample = loadmat('spamTest.mat')
# print(spamTest_sample)

X_test = spamTest_sample['Xtest'].T # (4000, 1899)
y_label_test = spamTest_sample['ytest'] # (4000, 1)
y_label_test = y_label_test.reshape(y_label_test.shape[0])

print(X_test.shape)
print(y_label_test.shape)

# print(y_label_test)

print(w.value.shape)
print(b.value)
print(w.value)
# accuracy = np.mean(np.round(y_label_test) == np.round(w.value.T@X_test + b.value))
# print(np.round(w.value.T@X.T + b.value))
print(accuracy_Fucked(y_label_test, X_test, w.value, b.value))
print(np.mean(y_label_test))
y_label2 = y_label/2+0.5
print(accuracy_Fucked(y_label2, X.T, w.value, b.value))

(1899, 1000)
(1000,)
(1899,)
[-0.00111174]
[2.75670684e-08 3.92975773e-08 2.56936352e-07 ... 1.14000738e-07
 1.10476546e-07 1.80124762e-07]
0.308
0.308
0.3285
